# Energy System Modelling - Tutorial V.3

Diesel can be produced via Fischer-Tropsch process from hydrogen and carbon dioxide. The necessary CO$_2$ can be captured from the atmosphere with direct air capture (DAC) and than be stored. 

Model the production of diesel via Fischer-Tropsch-process in PyPSA.  Build a pypsa model including transport and electricity demand. Add a store for hydrogen, gas and CO$_2$. Electricity can be generated with an Open-Cycle-gas turbine (OCGT) and for transport diesel cars can be used. All the diesel used during the investigated time period should also be produced via Fischer-Tropsch.

This jupyter notebook is partially filled out. Parts marked with ```??``` need to be filled out

In [147]:
import pypsa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('bmh')
%matplotlib inline

__(a) Read the demand (unit MWh) for electricity and transport from the csv file__ 

In [148]:
demand = pd.read_csv(??, index_col=0, parse_dates=[0])

Plot the demand for electricity and transport.

In [174]:
??

__How do you explaing the shape of the transport demand? (write your thoughts here)__

??

__(b) What is the meaning of ```e_cyclic```, ```e_initial``` and ```e_min_pu``` for the component store in the PyPSA syntax? (look in the [pypsa documentation]("https://pypsa.readthedocs.io/en/latest/components.html#store"), write your thoughts here)__

??

__(c) Create a pypsa network to model the production of synthetic fuels. To do so, allow multi-links__

Allow pypsa component link to have multiple out- and inputs (see [pypsa documentation](https://pypsa.org/doc/components.html#link-with-multiple-outputs-or-inputs))

In [150]:
override_component_attrs = pypsa.descriptors.Dict({k : v.copy() for k,v in pypsa.components.component_attrs.items()})
override_component_attrs["Link"].loc["bus2"] = ["string",np.nan,np.nan,"2nd bus","Input (optional)"]
override_component_attrs["Link"].loc["bus3"] = ["string",np.nan,np.nan,"3rd bus","Input (optional)"]
override_component_attrs["Link"].loc["efficiency2"] = ["static or series","per unit",1.,"2nd bus efficiency","Input (optional)"]
override_component_attrs["Link"].loc["efficiency3"] = ["static or series","per unit",1.,"3rd bus efficiency","Input (optional)"]
override_component_attrs["Link"].loc["p2"] = ["series","MW",0.,"2nd bus output","Output"]
override_component_attrs["Link"].loc["p3"] = ["series","MW",0.,"3rd bus output","Output"]

In [151]:
n = pypsa.Network(override_component_attrs=override_component_attrs)

Set the snapshots as the index of the demand time series


In [175]:
n.set_snapshots(??)

SyntaxError: invalid syntax (<ipython-input-175-6516f4f1eb6b>, line 1)

Add a bus and the corresponding load for the carriers transport, electricity

In [153]:
# electric bus + load
n.add("Bus",
      "electric")

n.add("Load",
      "electric",
      bus="electric",
      p_set=demand["electricity"])

# transport bus + load 
n.add("Bus",
      "transport")

n.add("Load",
      "transport",
      bus=??,
      p_set=??)


Add a bus for hydrogen, gas and diesel and a corresponding store


In [154]:
# hydrogen 
n.add("Bus",
      "hydrogen")

n.add("Store",
      "hydrogen",
      bus="hydrogen",
      e_nom=3e5,
      e_initial=3e5,
      marginal_cost=??)

# gas
n.add("Bus",
      "gas")

n.add("Store",
      "gas",
      bus=??,
      e_nom=??,
      e_initial=??,
      marginal_cost=??)

# diesel
n.add("Bus",
      "diesel")

n.add("Store",
      "diesel",
      bus="diesel",
      e_cyclic=??,
      e_nom_extendable=True)


The CO2 can be stored (for example in [underground caverns](https://en.wikipedia.org/wiki/Carbon_capture_and_storage#Geological_storage)). To model CO2 storage we have to keep track of the CO2
which is emitted to the atmosphere. Therefore we add a bus and a storage for the CO2 in the atmosphere 
and the stored CO2. The storage energy in our model can also become negative, e.g. if more CO2 is removed from the atmosphere than what there was initially. The storage capacity should be optimised.

In [155]:
# minus sign because opposite to how fossil fuels used:
# CH4 burning puts CH4 down, atmosphere up
n.add("Carrier",
      "co2",
      co2_emissions=-1.)

# CO2 in the atmosphere
n.add("Bus",
      "co2 atmosphere",
      carrier="co2")

n.add("Store",
      "co2 atmosphere",
      bus="co2 atmosphere",
      e_min_pu=??,
      e_nom_extendable=True)

# CO2 stored
n.add("Bus",
      "co2 stored")

n.add("Store",
      "co2 stored",
      bus="co2 stored",
      e_min_pu=??,
      capital_cost=??,
      e_nom_extendable=??,)

Add the possibility of [Direct Air Capture (DAC)](https://en.wikipedia.org/wiki/Direct_air_capture) of CO2 by adding a PyPSA link. The CO2 is captured from the atmosphere and stored.

In [156]:
# Direct Air Capture (DAC)
n.add("Link",
      "DAC",
      bus0="co2 atmosphere",
      bus1=??,
      efficiency=1,
      marginal_cost=??,
      p_nom_extendable=True)

__Why do we need to keep track of the CO2 by adding a bus and store for the CO2 in the atmosphere?__

??

Add diesel cars to satisfy the transport demand using a multi-link

In [157]:
# diesel car
n.add("Link",
      "diesel car",
      bus0="diesel",
      bus1=??,
      bus2="co2 atmosphere",
      efficiency=1.,
      efficiency2=1.,
      p_nom_extendable=True)

Add a open-cyclce gas turbine (OCGT) to satisfy the electricity demand

In [158]:
# OCGT
n.add("Link",
      "OCGT",
      bus0=??,
      bus1="electric",
      bus2="co2 atmosphere",
      efficiency=0.5,
      efficiency2=1,
      capital_cost=??,
      marginal_cost=??,
      p_nom_extendable=True)

Diesel can be produced from H2 and CO2 via [Fischer-Tropsch](https://en.wikipedia.org/wiki/Fischer%E2%80%93Tropsch_process).
Add a link which represents this process

In [159]:
n.add("Link",
      "Fischer-Tropsch",
      bus0=??,
      bus1="diesel",
      bus2="co2 stored",
      efficiency=1.,
      efficiency2=-1,
      capital_cost=??,
      p_nom_extendable=True)

__(d) Run an investment optimisation__

In [176]:
??

__(d)(i) Show that the total amount of CO2 in the system stays constant__

In [178]:
??

__(d)(ii) Plot the energy difference in the plots. You can use the pandas function ```diff()```__

In [177]:
??

__(d)(iii) What are the total system costs?__

In [179]:
??

__(e) Now add an constraint for the CO2 content in the atmosphere in the last time step to zero and rerun the investment optimisation (see also in PyPSA [documentation]("https://pypsa.readthedocs.io/en/latest/components.html#global-constraints") ```GlobalConstraint```)__

In [ ]:
limit = ??
n.add("GlobalConstraint",
      "co2_limit",
      sense="<=",
      carrier_attribute="co2_emissions",
      constant=limit)

In [180]:
# run investment optimisation
??

__How large are the total system costs now?__

In [181]:
??

__How much CO2 storage capacity is necessary?__

In [182]:
??

__(f) Add electric cars to the problem (using a PyPSA "Link" and the given efficiency) and solve the optimisation problem again. How large is the share of electric cars?__

In [ ]:
# electric car
n.add("Link",
      "electric car",
      bus0=??,
      bus1=??,
      efficiency=0.9,
      p_nom_extendable=True)

In [184]:
# run optimisation
??
# check link capacity

__(g) Now add the possibility to generate electricity with an onshore wind turbine. The time serie for the wind turbine is given in the data folder (wind_time_series.csv) and run the optimisation again. How large is the share of electric cars now?__

In [185]:
wind_timeseries = pd.read_csv(??)

n.add("Generator",
      "onwind",
      bus=??,
      p_nom_extendable=True,
      capital_cost=??,
      p_max_pu=??)

SyntaxError: invalid syntax (<ipython-input-185-4fa908a003e5>, line 1)

In [186]:
# run investment optimisation
??

In [187]:
# compare share of electric and diesel cars
??